# Practical Exam Sample: Pet Supplies

PetMind is a retailer of products for pets. They are based in the United States.

PetMind sells products that are a mix of luxury items and everyday items. Luxury items include toys. Everyday items include food.

The company wants to increase sales by selling more products for some animals repeatedly. 

They have been testing this approach for the last year. 

They now want a report on how repeat purchases impact sales. 

## Data

The data is available in the table `pet_supplies`.

The dataset contains the sales records in the stores last year. 

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|product_id | Nominal. The unique identifier of the product. </br>Missing values are not possible due to the database structure.|
| category | Nominal. The category of the product, one of 6 values (Housing, Food, Toys, Equipment, Medicine, Accessory). </br>Missing values should be replaced with “Unknown”. |
| animal | Nominal. The type of animal the product is for. One of Dog, Cat, Fish, Bird. </br>Missing values should be replaced with “Unknown”. |
| size | Ordinal. The size of animal the product is for. Small, Medium, Large. </br>Missing values should be replaced with “Unknown”.|
| price | Continuous. The price the product is sold at. Can be any positive value, round to 2 decimal places. </br>Missing values should be replaced with the overall median price. |
| sales | Continuous. The value of all sales of the product in the last year. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median sales. |
| rating | Discrete. Customer rating of the product from 1 to 10. </br>Missing values should be replaced with 0. |
| repeat_purchase | Nominal. Whether customers repeatedly buy the product (1) or not (0). </br>Missing values should be removed. |

# Task 1

From taking a quick look at the data, you are pretty certain it isn't quite as it should be. You need to make sure all of the data is clean before you start your analysis. The table below shows what the data should look like. 

Write a query to return a table that matches the description provided.

Do not update the original table. 

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|product_id | Nominal. The unique identifier of the product. </br>Missing values are not possible due to the database structure.|
| category | Nominal. The category of the product, one of 6 values (Housing, Food, Toys, Equipment, Medicine, Accessory). </br>Missing values should be replaced with “Unknown”. |
| animal | Nominal. The type of animal the product is for. One of Dog, Cat, Fish, Bird. </br>Missing values should be replaced with “Unknown”. |
| size | Ordinal. The size of animal the product is for. Small, Medium, Large. </br>Missing values should be replaced with “Unknown”.|
| price | Continuous. The price the product is sold at. Can be any positive value, round to 2 decimal places. </br>Missing values should be replaced with 0. |
| sales | Continuous. The value of all sales of the product in the last year. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median sales. |
| rating | Discrete. Customer rating of the product from 1 to 10. </br>Missing values should be replaced with 0. |
| repeat_purchase | Nominal. Whether customers repeatedly buy the product (1) or not (0). </br>Missing values should be removed. |

In [16]:
SELECT *
FROM pet_supplies
Limit 100; 

,product_id,category,animal,size,price,sales,rating,repeat_purchase
0,1,Food,Bird,large,51.1,1860.62,7.0,1
1,2,Housing,Bird,MEDIUM,35.98,963.60,6.0,0
2,3,Food,Dog,medium,31.23,898.30,5.0,1
3,4,Medicine,Cat,small,24.95,982.15,6.0,1
4,5,Housing,Cat,Small,26.18,832.63,7.0,1
...,...,...,...,...,...,...,...,...
95,96,Housing,Cat,MEDIUM,21.14,617.64,7.0,1
96,97,Accessory,Bird,LARGE,42.99,1417.24,5.0,0
97,98,Equipment,Cat,Small,27.95,1026.77,6.0,1
98,99,Equipment,Dog,Large,38,1447.26,5.0,0


In [15]:
-- check if price and sales is numeric
SELECT 
    column_name, 
    data_type 
FROM 
    information_schema.columns 
WHERE 
    table_name = 'pet_supplies' 
    AND column_name IN ('price', 'sales', 'rating');

,column_name,data_type
0,rating,integer
1,sales,double precision
2,price,text


In [7]:
SELECT price, COUNT(*)
FROM pet_supplies
GROUP BY price
order by price

,price,count
0,12.85,1
1,12.89,1
2,13.02,1
3,13.13,1
4,14.89,1
...,...,...
702,54.03,1
703,54.05,1
704,54.07,2
705,54.16,1


In [11]:
-- check for non null and uniqueness in primary key product_id
SELECT COUNT(*) AS missing
FROM pet_supplies
WHERE product_id IS NULL;

,missing
0,0


In [13]:
-- if there are duplicates, count >1 would appear at top
SELECT product_id, COUNT(*)
FROM pet_supplies
GROUP BY product_id
ORDER BY product_id DESC

,product_id,count
0,1500,1
1,1499,1
2,1498,1
3,1497,1
4,1496,1
...,...,...
1495,5,1
1496,4,1
1497,3,1
1498,2,1


In [17]:
SELECT 
  product_id,
  COALESCE(NULLIF(REPLACE(category, '-', ''), ''), 'Unknown') AS category,
  COALESCE(animal,'Unknown') as animal,
  COALESCE(INITCAP(size),'Unknown') as size,
  CASE 
    WHEN price = 'unlisted' THEN 0 
    ELSE ROUND(price :: Numeric,2) 
  END as price,
  COALESCE(
	  ROUND(sales :: Numeric ,2), 
	  ROUND((SELECT PERCENTILE_DISC(0.5) WITHIN GROUP (ORDER BY CAST(sales AS DECIMAL(10, 2))) FROM    pet_supplies), 2)) AS sales,
  COALESCE(rating,0) as rating,
  repeat_purchase
FROM 
  pet_supplies

,product_id,category,animal,size,price,sales,rating,repeat_purchase
0,1,Food,Bird,Large,51.10,1860.62,7,1
1,2,Housing,Bird,Medium,35.98,963.60,6,0
2,3,Food,Dog,Medium,31.23,898.30,5,1
3,4,Medicine,Cat,Small,24.95,982.15,6,1
4,5,Housing,Cat,Small,26.18,832.63,7,1
...,...,...,...,...,...,...,...,...
1495,1496,Food,Bird,Medium,40.91,1203.36,5,0
1496,1497,Equipment,Dog,Small,32.89,1099.97,5,0
1497,1498,Food,Fish,Small,25.79,724.02,3,0
1498,1499,Medicine,Fish,Small,19.94,658.66,5,1


# Task 2

You want to show whether sales are higher for repeat purchases for different animals. You also want to give a range for the sales.

Write a query to return the `animal`, `repeat_purchase` indicator and the `avg_sales`, along with the `min_sales` and `max_sales`. All values should be rounded to whole numbers. 

You should use the original `pet_supplies` data for this task. 

In [4]:
SELECT 
  animal, 
  repeat_purchase, 
  ROUND(AVG(sales :: NUMERIC),0) AS avg_sales ,
  ROUND(MIN(sales :: NUMERIC),0) AS min_sales , 
  ROUND(max(sales :: NUMERIC),0) AS max_sales
FROM 
  pet_supplies
GROUP BY 
  animal, repeat_purchase

,animal,repeat_purchase,avg_sales,min_sales,max_sales
0,Fish,1,693,287,1301
1,Bird,0,1380,858,2255
2,Dog,0,1084,574,1795
3,Dog,1,1038,574,1797
4,Cat,0,1035,512,1730
5,Bird,1,1408,853,2256
6,Fish,0,705,288,1307
7,Cat,1,998,512,1724


# Task 3

The management team want to focus on efforts in the next year on the most popular pets - cats and dogs - for products that are bought repeatedly. 

Write a query to return the `product_id`, `sales` and `rating` for the relevant products.

You should use the original `pet_supplies` data for this task.

In [21]:
SELECT 
  product_id , sales , rating 
FROM 
  pet_supplies
WHERE 
  product_id IN (
	  SELECT product_id
      FROM pet_supplies
      WHERE 
	      (animal = 'Cat' OR animal ='Dog')
          AND repeat_purchase = 1
  )
ORDER BY 
  sales DESC;

,product_id,sales,rating
0,518,1797.02,7.0
1,280,1795.77,5.0
2,728,1793.71,6.0
3,20,1792.63,7.0
4,946,1788.28,8.0
...,...,...,...
547,81,518.45,6.0
548,708,516.87,4.0
549,718,514.68,4.0
550,1423,512.64,6.0
